# Языковая модель на RNN

## Реккурентные нейронные сети (RNN)

Исходная последовательность:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

Для каждого входного значения $x_{1:i}$ получаем на выходе $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

Для всей последовательности $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ - рекурсивная функция активации, зависящая от двух параметров: $x_i$ и $s_{i-1}$ (вектор предыдущего состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- конкатенация $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

Построим языковую модель на основе RNN с помощью pytorch

In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные

In [3]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [4]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [5]:
trn_ds.lines[1]

'aardonyx'

In [6]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [7]:
trn_ds.vocab_size

29

In [8]:
x, y = trn_ds[1]

In [9]:
x.shape

torch.Size([9, 29])

In [10]:
y.shape

torch.Size([9])

Опишем модель, функцию потерь и алгоритм оптимизации

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(h)
        return h, y

In [12]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

![rnn](images/dinos3.png)

In [26]:
def sample(model):
    model.eval()
    end_char_idx = trn_ds.ch_to_idx['>'] # чтобы понимать, когда слово кончилось
    start_char_idx = trn_ds.ch_to_idx['<'] # seed для начала слова
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        
        # "сгенерирвали" начало слова
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != end_char_idx and len(indices) != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float() # нетривиальное решение: подаём не всегда то же, что выдали
            
            predicted_char_idx = idx
            
        
        if len(indices) == 50:
            indices.append(newline_idx)
    return indices

In [27]:
def print_sample(sample_idxs):
    print(''.join([trn_ds.idx_to_ch[x] for x in sample_idxs]))

Обучим получившуюся модель

In [28]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [29]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [30]:
train(model, loss_fn, optimizer, epochs = 10)

Epoch:1
<ataa>
<pudrosaurus>
<shurocaurus>
<seuaasamrus>
<hauaoturus>
<seltskvcurus>
<ipalturus>
<brjandsaurus>
<wrueosaurus>
<ysiposaurus>
<apnrakojangsnaurus>
<ptgrosaurus>
<snmtrbipaurus>
<tahrobaurus>
<scaviuaurus>

Epoch:2
<sturus>
<stainamsaurus>
<shamesheatdaurus>
<sonersaurus>
<tanibaurus>
<ajpiclltoryxcis>
<uaurtraurus>
<lrpuaaurus>
<smsanopcor>
<gnauegusiurus>
<eoreyaurus>
<imaesaratops>
<aicomaurus>
<ptgtposaurus>
<lsoucaurus>

Epoch:3
<sltagssaurus>
<aubitmtiousaurus>
<snrotaurus>
<arcoadssaurus>
<ascgvsaurus>
<erilyosrustus>
<esaresatopa>
<ctliceurus>
<btrrogaurus>
<pttankdaurus>
<aauraraurus>
<amuptaurus>
<pslysaurus>
<lcdocaurps>
<lbhhogauras>

Epoch:4
<nvbrtsaurus>
<tmohuberatops>
<tcdrnaen>
<gecrdauosaxtor>
<rrkytaurus>
<laapbsaurus>
<anbooaurus>
<puipounaurus>
<lpurasaurus>
<grcatocaurus>
<tciuougsiurus>
<iwtnossirus>
<scphycaurus>
<sngasaurus>
<stpmiosaurus>

Epoch:5
<tbilbduaurus>
<snaidourus>
<lurtcsiurus>
<jytosusars>
<gtatotaurus>
<cunicaurus>
<aushiersaurus>
<tb

### Task 2.
Rewrite the sampling function so that pangrams (words that contain each character of the alphabet only once)

### Task 3.
Rewrite the sampling function so that is it is possible to change the sampling temperature

### Task 4.
Implement the beam search for sampling

# Reference

1. Sampling in  RNN: https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
2. Coursera course (main source): https://github.com/furkanu/deeplearning.ai-pytorch/tree/master/5-%20Sequence%20Models
3. Coursera course (main source): https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
4. LSTM: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
5. Туториал по модулю LSTM в pytorch: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html